<div style="text-align: right">
Steven Esposito<br>
CSI 709<br>
11 Sep 2024
</div>

# Imports/Setup

In [1]:
import pandas as pd
import numpy as np
import luhn
import requests
import bs4 as bs

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sesposito/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
val_n = 4
val_k = 5
val_quant_lower = 0.25
val_quant_upper = 0.85

In [3]:
def scrape_page_wikipedia(val_url):
    html_scraped_text = requests.get(val_url)
    html_text_parsed = bs.BeautifulSoup(html_scraped_text.content,'lxml')
    html_text_content = html_text_parsed.find(id='content')
    html_text_paragraphs = html_text_content.find_all('p')
    val_text_formatted = ''
    for val in html_text_paragraphs:
        val_text_formatted += val.text
    return(val_text_formatted)

# Examples
## Request NLP Wikipedia Summary
### Use Luhn Score

| Parameter | Value (English) | Value (Set) |
| :-: | :-: | :-: |
| Stop Words Removed from Text| No | False |
| Counting Method | Luhn Counting | True |
| Stop Words Removed from Score | No | False |
| Aggregation Function | Luhn Algorithm | None |

In [4]:
%%time
val_text = scrape_page_wikipedia(val_url='https://en.wikipedia.org/wiki/natural_language_processing')
df_sents,df_words = luhn.luhn_tokenize(val_text=val_text,is_remove_stopwords=False)
df_words_scored = luhn.luhn_calc_signifcance_words(df=df_words,is_use_luhn_tf=True,is_remove_stopwords=False)
val_sig_lower,val_sig_upper = luhn.luhn_word_freq_cutoffs(df=df_words_scored,val_lower=val_quant_lower,val_upper=val_quant_upper)
df_words_scored['score'] = [luhn.luhn_calc_word_score(val_sig=x,val_lower=val_sig_lower,val_upper=val_sig_upper) for x in df_words_scored['significance']]
df_sentences_scored = luhn.luhn_calc_sentence_score_all(df=df_words_scored,val_num_apart=val_n,func_summary=None)
vec_scores,vec_sents = luhn.luhn_summarize(df_sentences=df_sents,df_scores=df_sentences_scored,val_num_sentences=val_k)
print('-'*50)
str_rtn = luhn.luhn_print_summary(vec_scores=vec_scores,vec_sentences=vec_sents)

Quantile Significance Lower = 1.00001
Quantile Significance Upper = 31.0
--------------------------------------------------
[57] Likewise, ideas of cognitive NLP are inherent to neural models multimodal NLP (although rarely made explicit)[58] and developments in artificial intelligence, specifically tools and technologies using large language model approaches[59] and new directions in artificial general intelligence based on the free energy principle[60] by British neuroscientist and theoretician at University College London Karl J. Friston. [15.6250] [9] In 2010, Tomáš Mikolov (then a PhD student at Brno University of Technology) with co-authors applied a simple recurrent neural network with a single hidden layer to language modelling,[10] and in the following years he went on to develop Word2vec. [14.2258] It is primarily concerned with providing computers with the ability to process data encoded in natural language and is thus closely related to information retrieval, knowledge repr

In [5]:
df_words_scored.head()

,word,id_sent,count,significance,score
0,natural,0,16,16,16
1,language,0,22,22,22
2,processing,0,11,11,11
3,nlp,0,15,15,15
4,is,0,13,13,13


In [6]:
df_sentences_scored.head()

,id_sent,score
40,40.0,15.625000
13,13.0,14.225806
1,1.0,12.970588
3,3.0,12.500000
18,18.0,12.250000


In [7]:
df_words_scored.loc[df_words_scored['id_sent']==1].head()

,word,id_sent,count,significance,score
14,it,1,3,3,3
15,is,1,13,13,13
16,primarily,1,1,1,0
17,concerned,1,1,1,0
18,with,1,16,16,16


### Experiment #1: Remove Stop Words

| Parameter | Value (English) | Value (Set) |
| :-: | :-: | :-: |
| Stop Words Removed from Text| Yes | True |
| Counting Method | Luhn Counting | True |
| Stop Words Removed from Score | No | False |
| Aggregation Function | Luhn Algorithm | None |

In [8]:
df_sents,df_words = luhn.luhn_tokenize(val_text=val_text,is_remove_stopwords=True)
df_words_scored = luhn.luhn_calc_signifcance_words(df=df_words,is_use_luhn_tf=True,is_remove_stopwords=False)
val_sig_lower,val_sig_upper = luhn.luhn_word_freq_cutoffs(df=df_words_scored,val_lower=val_quant_lower,val_upper=val_quant_upper)
df_words_scored['score'] = [luhn.luhn_calc_word_score(val_sig=x,val_lower=val_sig_lower,val_upper=val_sig_upper) for x in df_words_scored['significance']]
df_sentences_scored = luhn.luhn_calc_sentence_score_all(df=df_words_scored,val_num_apart=val_n,func_summary=None)
vec_scores,vec_sents = luhn.luhn_summarize(df_sentences=df_sents,df_scores=df_sentences_scored,val_num_sentences=val_k)
print('-'*50)
str_rtn = luhn.luhn_print_summary(vec_scores=vec_scores,vec_sentences=vec_sents)

Quantile Significance Lower = 1.00001
Quantile Significance Upper = 9.0
--------------------------------------------------
Machine learning approaches, which include both statistical and neural networks, on the other hand, have many advantages over the symbolic approach:  Although rule-based systems for manipulating symbols were still in use in 2020, they have become mostly obsolete with the advance of LLMs in 2023. [13.5000] In the 2010s, representation learning and deep neural network-style (featuring many hidden layers) machine learning methods became widespread in natural language processing. [8.0667] Since 2015,[22] the statistical approach has been replaced by the neural networks approach, using semantic networks[23] and word embeddings to capture semantic properties of words. [7.5789] [9] In 2010, Tomáš Mikolov (then a PhD student at Brno University of Technology) with co-authors applied a simple recurrent neural network with a single hidden layer to language modelling,[10] and 

### Experiment #2: Use Mean Score

| Parameter | Value (English) | Value (Set) |
| :-: | :-: | :-: |
| Stop Words Removed from Text| No | False |
| Counting Method | Luhn Counting | True |
| Stop Words Removed from Score | No | False |
| Aggregation Function | Mean | np.mean |

In [9]:
df_sents,df_words = luhn.luhn_tokenize(val_text=val_text,is_remove_stopwords=False)
df_words_scored = luhn.luhn_calc_signifcance_words(df=df_words,is_use_luhn_tf=True,is_remove_stopwords=False)
val_sig_lower,val_sig_upper = luhn.luhn_word_freq_cutoffs(df=df_words_scored,val_lower=val_quant_lower,val_upper=val_quant_upper)
df_words_scored['score'] = [luhn.luhn_calc_word_score(val_sig=x,val_lower=val_sig_lower,val_upper=val_sig_upper) for x in df_words_scored['significance']]
df_sentences_scored = luhn.luhn_calc_sentence_score_all(df=df_words_scored,val_num_apart=val_n,func_summary=np.mean)
vec_scores,vec_sents = luhn.luhn_summarize(df_sentences=df_sents,df_scores=df_sentences_scored,val_num_sentences=val_k)
print('-'*50)
str_rtn = luhn.luhn_print_summary(vec_scores=vec_scores,vec_sentences=vec_sents)

Quantile Significance Lower = 1.00001
Quantile Significance Upper = 31.0
--------------------------------------------------
It is primarily concerned with providing computers with the ability to process data encoded in natural language and is thus closely related to information retrieval, knowledge representation and computational linguistics, a subfield of linguistics. [6.7941] Major tasks in natural language processing are speech recognition, text classification, natural-language understanding, and natural-language generation. [6.6667] Natural language processing (NLP) is an interdisciplinary subfield of computer science and artificial intelligence. [6.5000] Typically data is collected in text corpora, using either rule-based, statistical or neural-based approaches in machine learning and deep learning. [6.1818] [57] Likewise, ideas of cognitive NLP are inherent to neural models multimodal NLP (although rarely made explicit)[58] and developments in artificial intelligence, specifical

### Experiment #3: Zero Score Stop Words

| Parameter | Value (English) | Value (Set) |
| :-: | :-: | :-: |
| Stop Words Removed from Text| No | False |
| Counting Method | Luhn Counting | True |
| Stop Words Removed from Score | Yes | True |
| Aggregation Function | Luhn Algorithm | None |

In [10]:
df_sents,df_words = luhn.luhn_tokenize(val_text=val_text,is_remove_stopwords=False)
df_words_scored = luhn.luhn_calc_signifcance_words(df=df_words,is_use_luhn_tf=True,is_remove_stopwords=True)
val_sig_lower,val_sig_upper = luhn.luhn_word_freq_cutoffs(df=df_words_scored,val_lower=val_quant_lower,val_upper=val_quant_upper)
df_words_scored['score'] = [luhn.luhn_calc_word_score(val_sig=x,val_lower=val_sig_lower,val_upper=val_sig_upper) for x in df_words_scored['significance']]
df_sentences_scored = luhn.luhn_calc_sentence_score_all(df=df_words_scored,val_num_apart=val_n,func_summary=None)
vec_scores,vec_sents = luhn.luhn_summarize(df_sentences=df_sents,df_scores=df_sentences_scored,val_num_sentences=val_k)
print('-'*50)
str_rtn = luhn.luhn_print_summary(vec_scores=vec_scores,vec_sentences=vec_sents)

Quantile Significance Lower = 1e-05
Quantile Significance Upper = 31.0
--------------------------------------------------
[57] Likewise, ideas of cognitive NLP are inherent to neural models multimodal NLP (although rarely made explicit)[58] and developments in artificial intelligence, specifically tools and technologies using large language model approaches[59] and new directions in artificial general intelligence based on the free energy principle[60] by British neuroscientist and theoretician at University College London Karl J. Friston. [26.5610] As an example, George Lakoff offers a methodology to build natural language processing (NLP) algorithms through the perspective of cognitive science, along with the findings of cognitive linguistics,[50] with two defining aspects: Ties with cognitive linguistics are part of the historical heritage of NLP, but they have been less frequently addressed since the statistical turn during the 1990s. [22.2308] [8] In 2003, word n-gram model, at th

### Experiment #4: Use TF Counting

$$tf=\frac{f_t}{\sum{f_t}}$$

| Parameter | Value (English) | Value (Set) |
| :-: | :-: | :-: |
| Stop Words Removed from Text| No | False |
| Counting Method | TF Counting | False |
| Stop Words Removed from Score | No | False |
| Aggregation Function | Mean | np.mean |

In [11]:
df_sents,df_words = luhn.luhn_tokenize(val_text=val_text,is_remove_stopwords=False)
df_words_scored = luhn.luhn_calc_signifcance_words(df=df_words,is_use_luhn_tf=False,is_remove_stopwords=False)
val_sig_lower,val_sig_upper = luhn.luhn_word_freq_cutoffs(df=df_words_scored,val_lower=val_quant_lower,val_upper=val_quant_upper)
df_words_scored['score'] = [luhn.luhn_calc_word_score(val_sig=x,val_lower=val_sig_lower,val_upper=val_sig_upper) for x in df_words_scored['significance']]
df_sentences_scored = luhn.luhn_calc_sentence_score_all(df=df_words_scored,val_num_apart=val_n,func_summary=np.mean)
vec_scores,vec_sents = luhn.luhn_summarize(df_sentences=df_sents,df_scores=df_sentences_scored,val_num_sentences=val_k)
print('-'*50)
str_rtn = luhn.luhn_print_summary(vec_scores=vec_scores,vec_sentences=vec_sents)

Quantile Significance Lower = 0.0008713264427217916
Quantile Significance Upper = 0.026701119724375538
--------------------------------------------------
It is primarily concerned with providing computers with the ability to process data encoded in natural language and is thus closely related to information retrieval, knowledge representation and computational linguistics, a subfield of linguistics. [6.7941] Major tasks in natural language processing are speech recognition, text classification, natural-language understanding, and natural-language generation. [6.6667] Natural language processing (NLP) is an interdisciplinary subfield of computer science and artificial intelligence. [6.5000] Typically data is collected in text corpora, using either rule-based, statistical or neural-based approaches in machine learning and deep learning. [6.1818] [57] Likewise, ideas of cognitive NLP are inherent to neural models multimodal NLP (although rarely made explicit)[58] and developments in artif

| Parameter | Value (English) | Value (Set) |
| :-: | :-: | :-: |
| Stop Words Removed from Text| No | False |
| Counting Method | TF Counting | False |
| Stop Words Removed from Score | No | False |
| Aggregation Function | Luhn Algorithm | None |

In [12]:
df_sents,df_words = luhn.luhn_tokenize(val_text=val_text,is_remove_stopwords=False)
df_words_scored = luhn.luhn_calc_signifcance_words(df=df_words,is_use_luhn_tf=False,is_remove_stopwords=False)
val_sig_lower,val_sig_upper = luhn.luhn_word_freq_cutoffs(df=df_words_scored,val_lower=val_quant_lower,val_upper=val_quant_upper)
df_words_scored['score'] = [luhn.luhn_calc_word_score(val_sig=x,val_lower=val_sig_lower,val_upper=val_sig_upper) for x in df_words_scored['significance']]
df_sentences_scored = luhn.luhn_calc_sentence_score_all(df=df_words_scored,val_num_apart=val_n,func_summary=None)
vec_scores,vec_sents = luhn.luhn_summarize(df_sentences=df_sents,df_scores=df_sentences_scored,val_num_sentences=val_k)
print('-'*50)
str_rtn = luhn.luhn_print_summary(vec_scores=vec_scores,vec_sentences=vec_sents)

Quantile Significance Lower = 0.0008713264427217916
Quantile Significance Upper = 0.026701119724375538
--------------------------------------------------
[57] Likewise, ideas of cognitive NLP are inherent to neural models multimodal NLP (although rarely made explicit)[58] and developments in artificial intelligence, specifically tools and technologies using large language model approaches[59] and new directions in artificial general intelligence based on the free energy principle[60] by British neuroscientist and theoretician at University College London Karl J. Friston. [15.6250] [9] In 2010, Tomáš Mikolov (then a PhD student at Brno University of Technology) with co-authors applied a simple recurrent neural network with a single hidden layer to language modelling,[10] and in the following years he went on to develop Word2vec. [14.2258] It is primarily concerned with providing computers with the ability to process data encoded in natural language and is thus closely related to informa

| Parameter | Value (English) | Value (Set) |
| :-: | :-: | :-: |
| Stop Words Removed from Text| No | False |
| Counting Method | TF Counting | False |
| Stop Words Removed from Score | Yes | True |
| Aggregation Function | Mean | np.mean |

In [13]:
df_sents,df_words = luhn.luhn_tokenize(val_text=val_text,is_remove_stopwords=False)
df_words_scored = luhn.luhn_calc_signifcance_words(df=df_words,is_use_luhn_tf=False,is_remove_stopwords=True)
val_sig_lower,val_sig_upper = luhn.luhn_word_freq_cutoffs(df=df_words_scored,val_lower=val_quant_lower,val_upper=val_quant_upper)
df_words_scored['score'] = [luhn.luhn_calc_word_score(val_sig=x,val_lower=val_sig_lower,val_upper=val_sig_upper) for x in df_words_scored['significance']]
df_sentences_scored = luhn.luhn_calc_sentence_score_all(df=df_words_scored,val_num_apart=val_n,func_summary=np.mean)
vec_scores,vec_sents = luhn.luhn_summarize(df_sentences=df_sents,df_scores=df_sentences_scored,val_num_sentences=val_k)
print('-'*50)
str_rtn = luhn.luhn_print_summary(vec_scores=vec_scores,vec_sentences=vec_sents)

Quantile Significance Lower = 1e-05
Quantile Significance Upper = 0.026701119724375538
--------------------------------------------------
[57] Likewise, ideas of cognitive NLP are inherent to neural models multimodal NLP (although rarely made explicit)[58] and developments in artificial intelligence, specifically tools and technologies using large language model approaches[59] and new directions in artificial general intelligence based on the free energy principle[60] by British neuroscientist and theoretician at University College London Karl J. Friston. [10.2712] As an example, George Lakoff offers a methodology to build natural language processing (NLP) algorithms through the perspective of cognitive science, along with the findings of cognitive linguistics,[50] with two defining aspects: Ties with cognitive linguistics are part of the historical heritage of NLP, but they have been less frequently addressed since the statistical turn during the 1990s. [10.1017] [17] Symbolic approac

## Request George Washington Wikipedia Summary

In [14]:
%%time
val_text = scrape_page_wikipedia(val_url='https://en.wikipedia.org/wiki/George_Washington')
df_sents,df_words = luhn.luhn_tokenize(val_text=val_text,is_remove_stopwords=False)
df_words_scored = luhn.luhn_calc_signifcance_words(df=df_words,is_use_luhn_tf=True,is_remove_stopwords=False)
val_sig_lower,val_sig_upper = luhn.luhn_word_freq_cutoffs(df=df_words_scored,val_lower=val_quant_lower,val_upper=val_quant_upper)
df_words_scored['score'] = [luhn.luhn_calc_word_score(val_sig=x,val_lower=val_sig_lower,val_upper=val_sig_upper) for x in df_words_scored['significance']]
df_sentences_scored = luhn.luhn_calc_sentence_score_all(df=df_words_scored,val_num_apart=val_n,func_summary=None)
vec_scores,vec_sents = luhn.luhn_summarize(df_sentences=df_sents,df_scores=df_sentences_scored,val_num_sentences=val_k)
print('-'*50)
str_rtn = luhn.luhn_print_summary(vec_scores=vec_scores,vec_sentences=vec_sents)

Quantile Significance Lower = 3.0
Quantile Significance Upper = 470.0
--------------------------------------------------
[38] Washington, impatient for an offensive against Fort Duquesne, was convinced Braddock would have granted him a royal commission and pressed his case in February 1756 with Braddock's successor as Commander-in-Chief, William Shirley, and again in January 1757 with Shirley's successor, Lord Loudoun. [31.1136] While his relationship with Washington would remain friendly, Washington's relationship with his Secretary of War Henry Knox deteriorated after rumors that Knox had profited from contracts for the construction of U.S. frigates which had been commissioned under the Naval Act of 1794 in order to combat Barbary pirates, forcing Knox to resign. [29.7609] Appointed by the Second Continental Congress as commander of the Continental Army in 1775, Washington led Patriot forces to victory in the American Revolutionary War and then served as president of the Constitution

## Request GMU Wikipedia Summary

In [15]:
%%time
val_text = scrape_page_wikipedia(val_url='https://en.wikipedia.org/wiki/George_Mason_University')
df_sents,df_words = luhn.luhn_tokenize(val_text=val_text,is_remove_stopwords=False)
df_words_scored = luhn.luhn_calc_signifcance_words(df=df_words,is_use_luhn_tf=True,is_remove_stopwords=False)
val_sig_lower,val_sig_upper = luhn.luhn_word_freq_cutoffs(df=df_words_scored,val_lower=val_quant_lower,val_upper=val_quant_upper)
df_words_scored['score'] = [luhn.luhn_calc_word_score(val_sig=x,val_lower=val_sig_lower,val_upper=val_sig_upper) for x in df_words_scored['significance']]
df_sentences_scored = luhn.luhn_calc_sentence_score_all(df=df_words_scored,val_num_apart=val_n,func_summary=None)
vec_scores,vec_sents = luhn.luhn_summarize(df_sentences=df_sents,df_scores=df_sentences_scored,val_num_sentences=val_k)
print('-'*50)
str_rtn = luhn.luhn_print_summary(vec_scores=vec_scores,vec_sentences=vec_sents)

Quantile Significance Lower = 2.0
Quantile Significance Upper = 127.0
--------------------------------------------------
8 in the nation for Freedom of Speech and protecting rights enshrined in the First Amendment to the United States Constitution according to the Foundation for Individual Rights and Expression (FIRE), FIRE also posits that the majority viewpoint of the student body leans politically liberal in the sense of modern liberalism in the United States,[138][139] although the political ideologies of libertarianism in the United States[140][141][142] and conservatism in the United States[143][144] are also visible on campus with the university stating that it strives for "comprehensive ideological balance," evidence including but not limited to the university being "home to both the Antonin Scalia Law School and the Jimmy and Rosalynn Carter School for Peace and Conflict Resolution," named after a conservative U.S. Supreme Court Justice (Antonin Scalia) politically appointed b

# Random Example

*Source: Scientific American*  
*Title: A New Quantum Cheshire Cat Thought Experiment Is Out of the Box*  
*Author: Manon Bischoff*

[A New Quantum Cheshire Cat Thought Experiment Is Out of the Box](https://www.scientificamerican.com/article/a-new-quantum-cheshire-cat-thought-experiment-is-out-of-the-box/)

In [16]:
val_text = '''Physicists seem to be obsessed with cats. James Clerk Maxwell, the father of electrodynamics, studied falling felines to investigate how they turned as they fell. Many physics teachers have used a cat’s fur and a hard rubber rod to explain the phenomenon of frictional electricity. And Erwin Schrödinger famously illustrated the strangeness of quantum physics with a thought experiment involving a cat that is neither dead nor alive.  So it hardly seems surprising that physicists turned to felines once again to name a newly discovered quantum phenomenon in a paper published in the New Journal of Physics in 2013. Their three-sentence study abstract reads, “In this paper we present a quantum Cheshire Cat. In a pre- and post-selected experiment we find the Cat in one place, and its grin in another. The Cat is a photon, while the grin is its circular polarization.”
The newfound phenomenon was one in which certain particle features take a different path from their particle—much like the smile of the Cheshire Cat in Alice’s Adventures in Wonderland, written by Lewis Carroll—a pen name of mathematician Charles Lutwidge Dodgson—and published in 1865. To date, several experiments have demonstrated this curious quantum effect. But the idea has also drawn significant skepticism. Critics are less concerned about the theoretical calculations or experimental rigor than they are about the interpretation of the evidence. “It seems a bit bold to me to talk about disembodied transmission,” says physicist Holger Hofmann of Hiroshima University in Japan. “Instead we should revise our idea of particles.”  Recently researchers led by Yakir Aharonov of Chapman University took the debate to the next level. Aharonov was a co-author of the first paper to propose the quantum Cheshire effect. Now, on the preprint server arXiv.org, he and his colleagues have posted a description of theoretical work that they believe demonstrates that quantum properties can move without any particles at all—like a disembodied grin flitting through the world and influencing its surroundings—in ways that bypass the critical concerns raised in the past.
Aharonov and his colleagues first encountered their quantum Cheshire cat several years ago as they were pondering one of the most fundamental principles of quantum mechanics: nothing can be predicted unambiguously. Unlike classical physics, the same quantum mechanical experiment can have different outcomes under exactly the same conditions. It is therefore impossible to predict the exact outcome of a single experiment—only its outcome with a certain probability. “Nobody understands quantum mechanics. It’s so counterintuitive. We know its laws, but we are always surprised,” says Sandu Popescu, a physicist at the University of Bristol in England, who collaborated with Aharonov on the 2013 paper and the new preprint.
But Aharonov was not satisfied with this uncertainty. So, since the 1980s, he has been exploring ways to investigate fundamental processes despite the probability-based nature of quantum mechanics. Aharonov—now age 92—employs an approach that involves intensively repeating an experiment, grouping results and then examining what came out before and after the experiment and relating these events to each other. “To do this, you have to understand the flow of time in quantum mechanics,” Popescu explains. “We developed a completely new method to combine information from measurements before and after the experiment.”
The researchers have stumbled across several surprises with this method—including their theoretical Cheshire cat. Their idea sounds simple at first: send particles through an optical tool called an interferometer, which causes each particle to move through one of two paths that ultimately merge again at the end. If the setup and measurements were carried out skillfully, Aharonov and his colleagues theorized, it could be shown that the particle traveled a path in the interferometer that differed from the path of its polarization. In other words, they claimed the property of the particle could be measured on one path even though the particle itself took the other—as if the grin and the cat had come apart.
Inspired by this theory, a team led by Tobias Denkmayr, then at the Vienna University of Technology, implemented the experiment with neutrons in a study published in 2014. The team showed that the neutral particles inside an interferometer followed a different path from that of their spin, a quantum mechanical property of particles similar to angular momentum: Denkmayr and his colleagues had indeed found evidence of the Cheshire cat theory. Two years later researchers led by Maximilian Schlosshauer of the University of Portland successfully implemented the same experiment with photons. The scientists saw evidence that the light particles took a different path in the interferometer than their polarization did.
But not everyone is convinced. “Such a separation makes no sense at all. The location of a particle is itself a property of the particle,” Hofmann says. “It would be more accurate to talk about an unusual correlation between location and polarization.” Last November Hofmann and his colleagues provided an alternative explanation based on widely known quantum mechanical effects.
And in another interpretation of the Cheshire cat results, Pablo Saldanha of the Federal University of Minas Gerais in Brazil and his colleagues argue that the findings can be explained with wave-particle duality. “If you take a different view, there are no paradoxes,” Saldanha says, “but all results can be explained with traditional quantum mechanics as simple interference effects.”
Much of the controversy surrounds the way in which particles’ properties and positions are detected in these experiments. Disturbing a particle could alter its quantum mechanical properties. For that reason, the photons or neutrons cannot be recorded inside the interferometer using an ordinary detector. Instead scientists must resort to a principle of weak measurement developed by Aharonov in 1988. A weak measurement makes it possible to scan a particle very lightly without destroying its quantum state. This comes at a price, however: the weak measurement result is extremely inaccurate. (Thus, these experiments must be repeated many times over, to compensate for the fact that each individual measurement is highly uncertain.)
In the quantum Cheshire cat experiments, a weak measurement is made along a path in the interferometer, the paths then merge, and the emerging particles are measured with an ordinary detector. Along one path of the interferometer, a weak measurement of the particle’s position can be taken and, along the other, its spin. Using detectors, physicists can more definitively characterize the particles that traveled through the interferometer and potentially reconstruct what occurred during the particle’s journey. For example, only certain particles will appear in certain detectors, helping the physicists piece together which path their neutron or photon previously took. According to Aharonov, Popescu and their colleagues, the Cheshire cat experiments ultimately reveal that the particle’s position can be confirmed on one path even as its polarization or spin was measured on the other.
Saldanha and his co-authors assert that it is impossible to make claims about quantum systems in the past given their measurements in the present. In other words, the photons and neutrons measured in the final detectors cannot tell us much about their previous trajectory. Instead the wave functions of particles passing through the paths of the interferometer could overlap, which would make it impossible to trace which path a particle had taken. “Ultimately, the paradoxical behaviors are related to the wave-particle duality,” Saldanha says. But in the papers that report evidence of the quantum Cheshire cat, he asserts, the findings “are processed in a sophisticated way that obscures this simpler interpretation.”
Hofmann, meanwhile, has stressed that the results will differ if you measure the system in a different way. This phenomenon is well-known in quantum physics: if, for example, you first measure the speed of a particle and then its position, the result can be different than it would be if you first measured the position of the same particle and then its speed. He and his colleagues therefore contend that Aharonov and his team’s conclusions were correct in themselves—that the particle moved along one path and the polarization followed the other—but that such differing paths do not apply simultaneously.
As Hofmann’s co-author Jonte Hance, also at Hiroshima University, told New Scientist, “It only looks like [the particle and polarization are] separated because you’re measuring one of the properties in one place and the other property in the other place, but that doesn’t mean that the properties are in one place and the other place, that means that the actual measuring itself is affecting it in such a way that it looks like it’s in one place and the other place.”
But these critiques are “missing the point,” Popescu says. He agrees that the work and reasoning put forward by Saldanha and Hofmann’s respective groups are correct—but adds that the best way to test any interpretation is to generate testable predictions from each. “As I understand it, there is no direct way to make predictions based on them,” Popescu says in reference to these alternative explanations. “They kind of have a very old-fashioned way of looking at things: there are contradictions, so you stop doing the math.”
With their recent preprint paper, Aharonov and Popescu, together with physicist Daniel Collins of the University of Bristol, have now described how a particle’s spin can move completely independently of the particle itself—without employing a weak measurement. In their new experimental setup, a particle is located in the left half of an elongated two-part cylinder that is sealed at the outer edges. Because of a highly reflective wall in the middle, the particle has a vanishingly small probability of tunneling through to the right-hand side of the cylinder. In their paper, the researchers provide a proof that even if the particle remains in the left-hand area in almost all cases, it should still be possible to measure a transfer of the particle’s spin at the right-hand outer wall. “It’s amazing, isn’t it?” Collins says. “You think the particle has a spin and the spin should stay with the particle. But the spin crosses the box without the particle.”
This approach would address several of the critical concerns raised thus far. The physicists don't need weak measurements. Nor do they need to group their experimental results to draw temporal conclusions. (That being said, grouping results would still improve the measurements, given that the angular momentum of the wall itself cannot be determined unambiguously because of the Heisenberg uncertainty principle.) But in this scenario, the only physical principles involved are conservation laws, such as the conservation of energy or the conservation of momentum and angular momentum. Popescu and Collins explain that they hope other groups will implement the experiment to observe the effects in the laboratory.
The new work has piqued Hofmann’s interest. “The scenario is exciting because the interaction between polarization and particle motion produces a particularly strong quantum effect that clearly contradicts the particle picture,” he says.
But he still does not see this as proof of disembodied (particle-free) spin transfer. “For me, this means, above all, that it is wrong to assume a measurement-independent reality,” Hofmann says. Instead quantum mechanics allows a particle’s residence to extend to the right-hand region of the cylinder, even if a residence in the left-hand region seems logically compelling. “I think it is quite clear to Aharonov, Collins and Popescu that the space in front of the wall is not really empty,” he adds.
Saldanha, meanwhile, still sees the researchers as overcomplicating what could be explained as traditional quantum interference effects. When discussing the particle’s very low probability of entering the right-hand side of the experimental setup, he explains, “we have to be careful about a ‘vanishingly small probability’ when we refer to waves.” The wave function of the particle could also expand into the right-hand side of the setup and thus influence the angular momentum of the wall. “The same predictions can be made without such dramatic conclusions,” he says.
In response to these critiques, Popescu says, “This is of course another way of thinking about it. The question is whether this interpretation is useful.” Regardless of which interpretation of the events is correct, the quantum Cheshire cat could enable new technological applications. For example, it could be used to transfer information or energy without moving a physical particle—whether made of matter or light.
For Popescu, however, the fundamental questions of physics play a more important role. “It all started when we thought about how time propagates in quantum mechanics,” he says. “And suddenly we were able to discover something fundamental about the laws of conservation.”
'''

In [17]:
%%time
df_sents,df_words = luhn.luhn_tokenize(val_text=val_text,is_remove_stopwords=False)
df_words_scored = luhn.luhn_calc_signifcance_words(df=df_words,is_use_luhn_tf=True,is_remove_stopwords=False)
val_sig_lower,val_sig_upper = luhn.luhn_word_freq_cutoffs(df=df_words_scored,val_lower=val_quant_lower,val_upper=val_quant_upper)
df_words_scored['score'] = [luhn.luhn_calc_word_score(val_sig=x,val_lower=val_sig_lower,val_upper=val_sig_upper) for x in df_words_scored['significance']]
df_sentences_scored = luhn.luhn_calc_sentence_score_all(df=df_words_scored,val_num_apart=val_n,func_summary=None)
vec_scores,vec_sents = luhn.luhn_summarize(df_sentences=df_sents,df_scores=df_sentences_scored,val_num_sentences=val_k)
print('-'*50)
str_rtn = luhn.luhn_print_summary(vec_scores=vec_scores,vec_sentences=vec_sents)

Quantile Significance Lower = 2.0
Quantile Significance Upper = 56.0
--------------------------------------------------
As Hofmann’s co-author Jonte Hance, also at Hiroshima University, told New Scientist, “It only looks like [the particle and polarization are] separated because you’re measuring one of the properties in one place and the other property in the other place, but that doesn’t mean that the properties are in one place and the other place, that means that the actual measuring itself is affecting it in such a way that it looks like it’s in one place and the other place.” But these critiques are “missing the point,” Popescu says. [54.4444] This phenomenon is well-known in quantum physics: if, for example, you first measure the speed of a particle and then its position, the result can be different than it would be if you first measured the position of the same particle and then its speed. [31.3913] According to Aharonov, Popescu and their colleagues, the Cheshire cat experiment

# Exhibit 2
*Source: The New York Times, September8, 1957, page E11*  
*Title: Chemistry Is Employed in a Search for New Methods toConquer Mental Illness*  
*Author: Robert K. Plumb*

In [18]:
val_text = '''By coincidence this weekend in New York City marks the end of the annual meeting of the American Psychological Association and the begining of 
the annual meeting of the American Chemical Society. Psychologists and chemists have never had so much in common as they now have in new studies of the chemical 
basis for human behavior. Exciting new finds in this field were also discussed last week in Iowa City, Iowa, at the annual meeting of the American 
Physiological Society and at Zurich, Switzerland, at the Second International Congress for Psychiatry. Two major recent developments have called the attention 
of chemists, physiologists, physicists and other scientists to mental diseases: It has been found that extremely minute quantities of chemicals can induce 
hallucinations and bizarre psychic disturbances in normal people, and mood-altering drugs (tranquilizers, for instance) have made long-institutionalized people 
amenable to therapy. Money to finance resreach on the physical factors in mental illness is being made available. Progress has 
been achieved toward the understanding of the chemistry of the brain. New goals are in sight. At the psychiatrists meeting in Zurich last week, four New York 
City physicians urged their colleagues to broaden their concept of "mental disease," and to probe more deeply into the chemistry and metabolism of the human 
body for answers to mental disorders and their prevention. Dr. Felix Marti-Ibanez and three brothers, Dr. Mortimer D. Sackler, 
Dr. Raymond R. Sackler and Dr. Arthur M. Sackler cited evidence that the blood chemistry of victims of schizophrenia is different from that of normal people. 
Perhaps multiple biological factors are responsible for this chemical change, they suggested. Mental disease is a "developmental process" and long duration of 
a disorder may result in "permanent alteration of anatomy and physiology," they said. They urged that trials of new drugs which affect the brain should be 
concentrated on complex studies of the mechanism of action of the drugs. The variety of substances capable of producing profound mental effects is a new 
armory of weapons for use in investigating biological mechanisms underlying mental disease, they said. The sources of behavioral disturbance are many and they 
may come from external as well as internal forces, the four reported. This concept has already proven practical, for instances, when it enabled psychiatrists 
to predict that the administration of ACTH and cortisone could produce psychosis. "It led some years ago to the development of a blood test which was 80 percent 
accurate in the identification of schizophrenic patients," they said. "It permitted us on physiologic grounds to deny that the psychoneuroses and the 
psychoses were lesser and greater degrees of the same disease process, and, in fact, to affirm that they represented opposite and even mutually exclusive 
directions of physiologic disturbances," they said. Chemicals now available should he used not only to bring relief to the mentally sick but also to uncover 
the biological mechanisms of the disease processes themselves. "Only then will the metabolic era mature and bring to fruition man's long hoped for salvation 
from the ravages of mental disease," they reported.
At the psychologist's meeting here, a technique for tracing electrical activity in specific portions of the animal brain was described by researchers from the 
University of California at Los Angeles. They reported that deep brain implants in cat brains were used to record electrical discharges created as the animals 
respond to stimulations to which they had been conditioned. In this way the California group reported, it is possible to track the sequence in which the brain 
brings its various parts into play in learning. Specific areas of memory in the brain may be located. Furthermore, the electrical pathways so traced out can 
be blocked temporarily by the use of chemicals. This poses new possibilities for studying brain chemistry changes in health and sickness and their alleviation, 
the California researchers emphasized. The new studies of brain chemistry have provided practical therapeutic results and tremendous encouragement to those who 
must care for mental patients. One evidence that knowledge in the interdisciplinary field is accumulating fast came last week in an announcement from Washington. 
This was the establishment by the National Institute of Mental Health of a clearing house of information on psychopharmacology. Literature in the field will be 
classified and coded so that staff members can answer a wide variety bf technical and scientific questions. People working in the field are invited to send three 
copies of papers or other material — even informal letters describing work they may have in progress to the Technical Information Unit of the center In 
Silver Spring, Md.'''
val_text = val_text.replace('\n','')

In [19]:
%%time
df_sents,df_words = luhn.luhn_tokenize(val_text=val_text,is_remove_stopwords=False)
df_words_scored = luhn.luhn_calc_signifcance_words(df=df_words,is_use_luhn_tf=True,is_remove_stopwords=False)
val_sig_lower,val_sig_upper = luhn.luhn_word_freq_cutoffs(df=df_words_scored,val_lower=val_quant_lower,val_upper=val_quant_upper)
df_words_scored['score'] = [luhn.luhn_calc_word_score(val_sig=x,val_lower=val_sig_lower,val_upper=val_sig_upper) for x in df_words_scored['significance']]
df_sentences_scored = luhn.luhn_calc_sentence_score_all(df=df_words_scored,val_num_apart=val_n,func_summary=None)
vec_scores,vec_sents = luhn.luhn_summarize(df_sentences=df_sents,df_scores=df_sentences_scored,val_num_sentences=val_k)
print('-'*50)
str_rtn = luhn.luhn_print_summary(vec_scores=vec_scores,vec_sentences=vec_sents)

Quantile Significance Lower = 1.00001
Quantile Significance Upper = 26.0
--------------------------------------------------
At the psychiatrists meeting in Zurich last week, four New York City physicians urged their colleagues to broaden their concept of "mental disease," and to probe more deeply into the chemistry and metabolism of the human body for answers to mental disorders and their prevention. [16.0000] Exciting new finds in this field were also discussed last week in Iowa City, Iowa, at the annual meeting of the American Physiological Society and at Zurich, Switzerland, at the Second International Congress for Psychiatry. [15.7500] Psychologists and chemists have never had so much in common as they now have in new studies of the chemical basis for human behavior. [11.6364] Two major recent developments have called the attention of chemists, physiologists, physicists and other scientists to mental diseases: It has been found that extremely minute quantities of chemicals can indu

# Potential Issues...

- Parsing Wikipedia with NLTK results in sentences that are sometimes more than one independent clause/sentence.
- Reproducing the results from the paper is complicated by the fact that exact values for *C* and *D* are not documented.